In [ ]:
import requests
import csv
import os
import pickle
import pandas as pd
import numpy as np
from zipfile import ZipFile
from matplotlib import pyplot as plt

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
def horizontal_barplot(df, rowcount, sort_value, sort_ascending, xcolumn, ycolumn, title='', 
                       xlabel='', ylabel='', invert_yaxis=True):
    fig, ax = plt.subplots(figsize=(15,8))
    plot_data = df.sort_values(sort_value, ascending=sort_ascending)[:rowcount]
    rects = ax.barh(plot_data[xcolumn], plot_data[ycolumn])

    for rect in rects:
        yloc = rect.get_y() + rect.get_height() / 2
        label = ax.annotate(str(rect.get_width()), xy=(rect.get_width(), yloc), xytext=(5, 0),
                            textcoords="offset points",
                            ha="left", va='center')
    if invert_yaxis: ax.invert_yaxis()

    ax.set_title(title)
    ax.set_xlabel(xlabel)    
    ax.set_ylabel(ylabel)   

    
def horizontal_driver_lines_plot(df, xcolumn, ycolumn, invert_yaxis=False, 
                                 title = '', xlabel='', ylabel='', 
                                 ymin=None, ymax=None):
    fig, ax = plt.subplots(figsize=(15,10))
    drivers = df['code'].unique()
    for driver in drivers:
        df[df.code == driver].plot(xcolumn, ycolumn, ax=ax, label=driver, 
                                   color=stats.get_driver_color(driver))
    ax.get_legend().remove()
    if invert_yaxis: ax.invert_yaxis()
    if ymin: 
      ax.set_ylim(ymin, ymax)
    else:
        ymax = int(df[ycolumn].max())+1
    ax.set_yticks(range(1, ymax, 1))
    xmax = int(df[xcolumn].max())+1
    ax.set_xticks(range(1, xmax, 2))
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)

    for line, name in zip(ax.lines, drivers):
        y = line.get_ydata()[-1]
        x = line.get_xdata()[-1]
        text = ax.annotate(name, xy=(x + 0.1, y), xytext=(0, 0), 
                           color=line.get_color(), textcoords="offset points")


In [ ]:
class F1Stats:
    
    zipfile=None
    cachefile=None
    dfs=None
    test={"A":"a", "B":"b"}
    # Geleend van FastF1
    driver_colors = {
        "BOT": "#900000", "ZHO": "#500000", "GAS": "#2b4562", "TSU": "#356cac", "ALO": "#0090ff", "OCO": "#70c2ff",
        "VET": "#006f62", "STR": "#25a617", "HUL": "#2f9b90", "LEC": "#dc0000", "SAI": "#ff8181", 
        "MAG": "#aaaaaa", "MSC": "#cacaca", "RIC": "#ff8700", "NOR": "#eeb370", "HAM": "#00d2be", "RUS": "#24ffff", 
        "VER": "#0600ef", "PER": "#716de2", "ALB": "#005aff", "LAT": "#012564"
    }
    
    points = [0, 25, 18, 15, 12, 10, 8, 6, 4, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    
    def __init__(self, zipdir='f1db_csv', zipfile='f1db.zip', cachefile='f1stats.pcl'):
        self.zipfile = os.path.join(zipdir, zipfile)
        self.cachefile = cachefile

    def download_data(self):
        url = 'http://ergast.com/downloads/f1db_csv.zip'
        r = requests.get(url, allow_redirects=True)
        open(self.zipfile, 'wb').write(r.content)
        # Open the zipfile
        zip_file = ZipFile(self.zipfile)
        # and read all files into a dictionary
        # key is filename (without .csv extension
        self.dfs = {text_file.filename[:-4]: pd.read_csv(zip_file.open(text_file.filename)).replace('\\N', '') \
                    for text_file in zip_file.infolist() if text_file.filename.endswith('.csv')} 
        print(self.dfs.keys())
        
    def save_data(self):
        with open(self.cachefile, 'wb') as handle:
            pickle.dump(self.dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    def load_data(self):
        with open(self.cachefile, 'rb') as handle:
            self.dfs = pickle.load(handle)
            
    def initialize(self, download=True):
        if download:
            self.download_data()
            self.cleanup_data()
            self.save_data()
        else:
            self.load_data()        

    def __column_strip_string(self, dfname, columnnames):
        print("Stripping string in \"{}\" for columns {}".format(dfname, columnnames))
        for c in columnnames:
            self.dfs[dfname][c] = self.dfs[dfname][c].str.strip()

    def __column_to_int(self, dfname, columnnames, default=0):
        print("Convert to int   in \"{}\" for columns {}".format(dfname, columnnames))
        for c in columnnames:
            self.dfs[dfname][c] = self.dfs[dfname][c].replace('', default).astype(int)

    def __column_to_float(self, dfname, columnnames, default=0):
        print("Convert to float in \"{}\" for columns {}".format(dfname, columnnames))
        for c in columnnames:
            self.dfs[dfname][c] = self.dfs[dfname][c].replace('', default).astype(float)

    def __column_to_datetime(self, dfname, datecolumn, timecolumn=None, newcolumn=None, onerror='ignore'):
        print("To datetime in      \"{}\" for columns {} and {} ".format(dfname, datecolumn, timecolumn))
        dropcolumns=[]
        if timecolumn:
            self.dfs[dfname][datecolumn] = self.dfs[dfname][datecolumn] + ' ' + self.dfs[dfname][timecolumn]
            dropcolumns.extend([timecolumn])
        targetcolumn = datecolumn
        if newcolumn:
            targetcolumn = newcolumn
            dropcolumns.extend([datecolumn])
        self.dfs[dfname][targetcolumn] = pd.to_datetime(self.dfs[dfname][datecolumn], errors=onerror)
        self.__drop_columns(dfname, dropcolumns)

    def __column_to_time(self, dfname, datecolumn, timecolumn=None, newcolumn=None, onerror='ignore'):
        self.__column_to_datetime(dfname, datecolumn, timecolumn, newcolumn, onerror)
        targetcolumn = datecolumn
        if newcolumn:
            targetcolumn = newcolumn
        self.dfs[dfname][targetcolumn] = self.dfs[dfname][targetcolumn].dt.time

    def __column_to_date(self, dfname, datecolumn, timecolumn=None, newcolumn=None, onerror='ignore'):
        self.__column_to_datetime(dfname, datecolumn, timecolumn, newcolumn, onerror)
        targetcolumn = datecolumn
        if newcolumn:
            targetcolumn = newcolumn
        self.dfs[dfname][targetcolumn] = self.dfs[dfname][targetcolumn].dt.date
        
    def __drop_columns(self, dfname, dropcolumns):
        print("Dropping columns in \"{}\" : {}".format(dfname, dropcolumns))
        self.dfs[dfname] = self.dfs[dfname].drop(columns=dropcolumns)
        
    def cleanup_data(self):
        # Races
        self.__column_strip_string("races", ["name"])
        self.__column_to_datetime("races", "date", "time")
        self.__column_to_datetime("races", "fp1_date", "fp1_time", "fp1")
        self.__column_to_datetime("races", "fp2_date", "fp2_time", "fp2")
        self.__column_to_datetime("races", "fp3_date", "fp3_time", "fp3")
        self.__column_to_datetime("races", "quali_date", "quali_time", "quali")
        self.__column_to_datetime("races", "sprint_date", "sprint_time", "sprint")
        self.__drop_columns("races", ["url"])
        # Results
        self.__column_to_int("results", ['position', 'number', 'fastestLap', 'rank'])
        self.__column_to_float("results", ['fastestLapSpeed'])
        self.__column_strip_string("results", ["positionText"])
        self.__column_to_time("results", "fastestLapTime")
        # Drivers
        self.__column_to_int("drivers", ['number'])
        self.__column_strip_string("drivers", ['driverRef', 'code', 'forename', 'surname', 'nationality'])
        self.__column_to_date("drivers", "dob")
        self.dfs['drivers']['name'] = self.dfs['drivers']['forename'] + ' ' + self.dfs['drivers']['surname']
        self.__drop_columns('drivers', ['url'])
        # Constructors
        self.__column_strip_string("constructors", ['constructorRef', 'name', 'nationality'])
        self.__drop_columns("constructors", ['url'])
        # Qualifying
        self.__column_to_time("qualifying", "q1")
        self.__column_to_time("qualifying", "q2")
        self.__column_to_time("qualifying", "q3")
        # Sprint results
        self.__column_to_int("sprint_results", ['position', 'number', 'fastestLap'])
        self.__column_strip_string("sprint_results", ["positionText"])
        self.__column_to_time("sprint_results", "fastestLapTime")
        # Driver standings
        self.__column_strip_string("driver_standings", ['positionText'])
        # Constructor standings
        self.__column_strip_string("constructor_standings", ['positionText'])
        # Lap times
        self.__column_to_time("lap_times", "time", onerror='coerce')
        # Pit stops
        self.__column_to_time("pit_stops", "time", onerror='coerce')

    def get_table(self, table):
        return self.dfs[table].copy()

    def get_race_results_simple(self):
        raceresults = pd.merge(self.dfs['results'][['resultId', 'raceId', 'driverId', 'constructorId', 
                                                    'grid','position', 'positionText']], 
                               self.dfs['races'][['raceId', 'year', 'round', 'name', 'date']]. \
                                    rename(columns={'name': 'race'}))
        raceresults = pd.merge(raceresults,
                               self.dfs['drivers'][['driverId', 'number', 'name', 'code']]. \
                                    rename(columns={'name': 'driver'}))
        raceresults = pd.merge(raceresults,
                               self.dfs['constructors'][['constructorId', 'name']]. \
                                    rename(columns={'name': 'constructor'}))
        raceresults = raceresults.drop(columns=['resultId'])
        raceresults = raceresults.sort_values(['year', 'round', 'position'])
        raceresults['points'] = raceresults['position'].apply(lambda x: self.points[x])
        return raceresults.copy()

    def get_race_results(self):
        raceresults = \
            pd.merge(
                     pd.merge(pd.merge(self.dfs['results'][['resultId', 'raceId', 'driverId', 'constructorId', 
                                                            'grid','position', 'positionText']], 
                                       self.dfs['races'][['raceId', 'year', 'round', 'name', 'date']]. \
                                               rename(columns={'name': 'race'})),
                              self.dfs['drivers'][['driverId', 'number', 'name', 'code']]. \
                                      rename(columns={'name': 'driver'})),
                     self.dfs['constructors'][['constructorId', 'name']].rename(columns={'name': 'constructor'})
                    )
        raceresults = raceresults.drop(columns=['resultId'])
        raceresults = raceresults.sort_values(['year', 'round', 'position'])
        raceresults['points'] = raceresults['position'].apply(lambda x: self.points[x])
        return raceresults.copy()
    
    def get_winners(self):
        winners = self.get_race_results()
        winners = winners[winners.position == 1]
        return winners.copy()

    def get_quali_results(self):
        qualiresults = \
            pd.merge(
                     pd.merge(pd.merge(self.dfs['qualifying'][['qualifyId', 'raceId', 'driverId', 'constructorId', 
                                                            'position']], 
                                       self.dfs['races'][['raceId', 'year', 'round', 'name', 'date']]. \
                                               rename(columns={'name': 'race'})),
                              self.dfs['drivers'][['driverId', 'number', 'forename', 'surname', 'name']]. \
                                      rename(columns={'name': 'driver'})),
                     self.dfs['constructors'][['constructorId', 'name']].rename(columns={'name': 'constructor'})
                    )
        qualiresults = qualiresults.drop(columns=['qualifyId', 'forename', 'surname'])
        qualiresults = qualiresults.sort_values(['year', 'round', 'position'])
        return qualiresults.copy()

    def get_pole_sitters(self):
        poles = self.get_quali_results()
        poles = poles[poles.position ==1]
        return poles

    def get_dnfs(self):
        dnfs = self.get_race_results()
        dnfs = dnfs[~dnfs['positionText'].str.isnumeric()]
        return dnfs.copy()
    
    def get_wdc_standing(self, year=None):
        standings = \
           self.dfs["driver_standings"]. \
            merge(self.dfs["races"][['raceId', 'year', 'round', 'name']].rename(columns={'name':'race'}), 
                  on='raceId') .\
            merge(self.dfs["drivers"][['driverId', 'name', 'code']])
        return standings[standings.year == year].copy() if year else standings.copy()

    def get_wcc_standing(self, year=None):
        standings = \
            stats.dfs["constructor_standings"]. \
                merge(stats.dfs["races"][['raceId', 'year', 'round', 'name']].rename(columns={'name':'race'}),
                      on='raceId') .\
                merge(stats.dfs["constructors"][['constructorId', 'name']])    
        return standings[standings.year == year].copy() if year else standings.copy()
    
    
    def get_wdc_champions(self):
        standings = self.get_wdc_standing()
        champions = standings[['year', 'round']].groupby('year').max().reset_index()
        champions['position'] = 1
        champions = champions.merge(
                        standings[['year', 'round', 'position', 'driverId', 'name', 'code', 'points', 'wins']])
        champions['perc_win'] =  (champions['wins'] / champions['round'] * 1000).astype(int) / 10
        return champions.copy()
    
    def get_laps_for_race(self, year, race):
        races = self.dfs['races']
        races = races[races.year == year]
        if isinstance(race,int):
            races = races[races["round"] == race]
        else:
            races = races[races["name"].str.contains(race)]
        raceID = races.iloc[0]['raceId']
        laps = self.dfs['lap_times'][self.dfs['lap_times']['raceId'] == raceID]
        laps = laps.merge(self.dfs["drivers"][['driverId', 'name', 'code']])
        laps = laps.merge(self.dfs['races'][['raceId', 'name']].rename(columns={'name':'event'}), on='raceId')
        return laps.copy()
    
    def get_driver_color(self, code):
        if code in set(self.dfs['drivers']['driverId'].unique()):
            code = self.dfs['drivers'][self.dfs['drivers']['driverId'] == code]['code'].values[0]
        if code in self.driver_colors:
            return self.driver_colors[code]
        else:
            return "#000000"
        
    def info(self, methods=True, tables=True):
        if methods:
            methodList = []
            for method_name in dir(F1Stats):
                try:
                    if callable(getattr(object, method_name)):
                        if not method_name.startswith('_'):
                            methodList.append(str(method_name))
                except Exception:
                    if not method_name.startswith('_'):
                        methodList.append(str(method_name))
            print('Methods')
            print('=======')
            for m in methodList:
                print(m)
            print()
        if tables:
            print('Tables')
            print('======')
            for key in stats.dfs.keys():
                print('{} '.format(key))
                for clm in stats.dfs[key].columns:
                    print('   {:22s}  - {}'.format(clm, stats.dfs[key][clm].dtype))

stats = F1Stats()
stats.initialize(download=False)
stats.dfs.keys()
stats.info(methods=True, tables=False)

In [ ]:
# # Get required dataframes
# races = stats.get_table("races")
# drivers = stats.get_table("drivers")
# constructors = stats.get_table("constructors")

# results = stats.get_table("results")
# constructor_results = stats.get_table("constructor_results")
# qualifying = stats.get_table("qualifying")
# sprint_results = stats.get_table("sprint_results")

# driver_standings = stats.get_table("driver_standings")
# constructor_standings = stats.get_table("constructor_standings")

# lap_times = stats.get_table("lap_times")
# pit_stops = stats.get_table("pit_stops")
# status = stats.get_table("status")

# Determine the number of consequetive wins

In [ ]:
winners = stats.get_winners()
winners['conseq_winner'] = (winners['year'] == winners['year'].shift(1)) & \
                           (winners['driverId'] == winners['driverId'].shift(1))
# Count consecutive occurences (does this for both True and False!)
winners["conseq_count"] = winners["conseq_winner"].groupby((winners["conseq_winner"] != winners["conseq_winner"]. \
                                                            shift()).cumsum()).cumcount()
winners["conseq_count"] = winners["conseq_count"] + 1
# # Only keep the True
winners.loc[~winners.conseq_winner, 'conseq_count'] = 1
winners['conseq_count2'] = winners['conseq_count'].shift(-1)
winners.loc[winners.conseq_count2 == 2, 'conseq_count'] = 1
# # Add first win to the count
winners.conseq_count= winners.apply(lambda row: row.conseq_count+1 if row.conseq_winner else row.conseq_count, axis=1)
winners = winners.drop(columns=['conseq_winner', 'conseq_count2'])
winners

In [ ]:
winners[((winners.conseq_count >= 6))]

In [ ]:
winners[winners.year == 1993]

# Determine number of wins per driver and constructor

In [ ]:
winners = stats.get_winners()
wins = winners[['driverId', 'driver', 'race']].groupby(['driverId', 'driver']).count().reset_index()
wins = wins.sort_values('race', ascending=False)

In [ ]:
horizontal_barplot(df=wins, rowcount=10, sort_value='race', sort_ascending=False, 
                   xcolumn='driver', ycolumn='race', invert_yaxis=True,
                   title='Drivers with most wins', 
                   xlabel='wins', ylabel='')

In [ ]:
wins = winners[['constructor', 'race']].groupby('constructor').count().reset_index().sort_values('race', ascending=False)

horizontal_barplot(df=wins, rowcount=10, sort_value='race', sort_ascending=False, 
                   xcolumn='constructor', ycolumn='race', invert_yaxis=True,
                   title='Wins per constructor', 
                   xlabel='wins', ylabel='')

In [ ]:
df = winners[['constructor', 'driver', 'race']].groupby(['constructor', 'driver']).count(). \
            sort_values('constructor')
df[df.race >= 15].sort_values(['constructor', 'race'], ascending=False)

# Drivers WDC standings over the season

In [ ]:
year=2022
standings = stats.get_wdc_standing(year)
standings

In [ ]:
horizontal_driver_lines_plot(df=standings, xcolumn='round', ycolumn='position', invert=True,
                             title='Championships standings ({})'.format(year),
                             xlabel='Round', ylabel='Position')

In [ ]:
horizontal_driver_lines_plot(df=standings, xcolumn='round', ycolumn='points', invert=False,
                             title='Championships points ({})'.format(year),
                             xlabel='Round', ylabel='Points')

# Number of wins per champion

In [ ]:
standings = stats.get_wdc_standing(None)

In [ ]:
champions = stats.get_wdc_champions()
champions.sort_values('perc_win').head(10)

In [ ]:
ax = champions.plot.bar('year', 'wins', figsize=(15,5))
ax.set_ylabel('Wins')
ax.set_title('Number of races won by champion')

In [ ]:
ax = champions.plot.bar('year', 'perc_win', figsize=(15,5))
ax.set_ylabel('Percentage')
ax.set_title('Percentage races won by champion')

# Most wins by drivers never being world champion

In [ ]:
winners = stats.get_winners()
wins_per_driver = winners[['driverId', 'year']].groupby('driverId').count().reset_index(). \
                            rename(columns={"year": "podiums"})
champs = stats.get_wdc_champions()

In [ ]:
not_champ = wins_per_driver[~wins_per_driver.driverId.isin(pd.unique(champs['driverId']))]
not_champ = not_champ.merge(stats.get_table('drivers'))[['driverId', 'name', 'podiums']]
not_champ = not_champ.sort_values('podiums', ascending=False)

In [ ]:
horizontal_barplot(df=not_champ, rowcount=10, sort_value='podiums', sort_ascending=False, 
                   xcolumn='name', ycolumn='podiums', 
                   title='Drivers with most wins without winning a championship', 
                   xlabel='wins', ylabel='')

# Most podiums  without champion

In [ ]:
podiums = stats.get_race_results()
podiums = podiums[podiums.position.between(1, 3)]
podiums_per_driver = podiums[['driverId', 'year']].groupby('driverId').count().reset_index(). \
                            rename(columns={"year": "podiums"})
champs = stats.get_wdc_champions()

In [ ]:
not_champ = podiums_per_driver[~podiums_per_driver.driverId.isin(pd.unique(champs['driverId']))]
not_champ = not_champ.merge(stats.get_table('drivers'))[['driverId', 'name', 'podiums']]
not_champ = not_champ.sort_values('podiums', ascending=False)

In [ ]:
horizontal_barplot(df=not_champ, rowcount=10, sort_value='podiums', sort_ascending=False, 
                   xcolumn='name', ycolumn='podiums', 
                   title='Drivers with most podiums without winning a championship', 
                   xlabel='podiums', ylabel='')

# Grandprix lap chart

In [ ]:
year = 2022
event = "Hungarian"
laps = stats.get_laps_for_race(year, event)


# Add startgrid for lap 0

# what about drivers finishing on a lap


In [ ]:
horizontal_driver_lines_plot(df=laps, xcolumn='lap', ycolumn='position', invert=True,
                             title='Lap chart for {} - {}'.format(year, laps['event'].iloc[0]),
                             xlabel='Lap', ylabel='Position')

# Compare points at a round with previous season

In [ ]:
year=2022
nw = stats.get_wdc_standing(year)
rnd = nw['round'].max()
nw = nw[nw['round'] == rnd][['position', 'code', 'name', 'points']].sort_values('points', ascending=False)
pv = stats.get_wdc_standing(year-1)
pv = pv[pv['round'] == rnd][['code', 'points']].sort_values('points', ascending=False)
pv = pv.rename(columns={"points": "prev_year"})
comp = pd.merge(nw, pv, on='code')
comp['diff'] = comp['points'] - comp['prev_year']
comp['color'] = 'green'
comp.loc[comp['diff'] < 0, 'color'] = 'red'

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
 
# Plotting the horizontal lines
rects = ax.hlines(y=comp.code, xmin=0, xmax=comp['diff'], color=comp['color'], alpha=0.4, linewidth=20)
ax.invert_yaxis()

for nm, vl in zip(comp.code, comp['diff']):
    label = ax.annotate(str(vl), xy=(vl, nm), xytext=(5 if vl > 0 else -30, 0),
                        textcoords="offset points",
                        ha="left", va='center')
ax.axvline(0, color='black')
ax.set_title('Difference in points after {} rounds between {} and {}'.format(rnd, year, year-1))
_ = ax.set_xlabel('Points difference')

In [ ]:
year=2022
nw = stats.get_wcc_standing(year)
rnd = nw['round'].max()
nw = nw[nw['round'] == rnd][['position', 'name', 'points']].sort_values('points', ascending=False)
pv = stats.get_wcc_standing(year-1)
pv = pv[pv['round'] == rnd][['name', 'points']].sort_values('points', ascending=False)
pv = pv.rename(columns={"points": "prev_year"})
comp = pd.merge(nw, pv, on='name')
comp['diff'] = comp['points'] - comp['prev_year']
comp['color'] = 'green'
comp.loc[comp['diff'] < 0, 'color'] = 'red'

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
 
# Plotting the horizontal lines
rects = ax.hlines(y=comp.name, xmin=0, xmax=comp['diff'], color=comp['color'], alpha=0.4, linewidth=20)
ax.invert_yaxis()

for nm, vl in zip(comp.name, comp['diff']):
    label = ax.annotate(str(vl), xy=(vl, nm), xytext=(5 if vl > 0 else -30, 0),
                        textcoords="offset points",
                        ha="left", va='center')
ax.axvline(0, color='black')
ax.set_title('Difference in points after {} rounds between {} and {}'.format(rnd, year, year-1))
_ = ax.set_xlabel('Points difference')

# Pole positions and pole+wins per driver

In [ ]:
winners = stats.get_winners()
poles = stats.get_pole_sitters()

pandw = pd.merge(winners.add_prefix('win_'), poles.add_prefix('pole_'), left_on='win_raceId', right_on='pole_raceId')
pandw = pandw[['win_raceId', 'win_year', 'win_round', 'win_driverId', 'win_driver', 'pole_driverId', 'pole_driver']]
pandw['pole_and_win'] = (pandw['win_driverId'] == pandw['pole_driverId'])

In [ ]:
pw = pandw[pandw.pole_and_win][['pole_driver', 'pole_driverId', 'win_raceId']].groupby(['pole_driver', 'pole_driverId']).count().reset_index().rename(columns={'win_raceId' : 'poles_and_wins'})
w = pandw[['pole_driver', 'pole_driverId', 'win_raceId']].groupby(['pole_driver', 'pole_driverId']).count().reset_index().rename(columns={'win_raceId' : 'poles'})
res = pd.merge(w, pw, on = 'pole_driverId').drop(columns=['pole_driver_y'])[['pole_driver_x', 'poles', 'poles_and_wins']]
res = res.rename(columns={'pole_driver_x': 'pole_driver'}).sort_values('poles', ascending=False)
res.head(10)

# Average finishing position

In [ ]:
year = 2022
results = stats.get_race_results()
results = results[results['year'] == year]
# DNF equals position 20
results.loc[results['position'] == 0, 'position'] = 20
#results = results[results['position'] > 0]
results[['driver', 'position']].groupby('driver').mean().sort_values('position')

# Places gained and lost

In [ ]:
year = 2022
rnd = 13
pgl = stats.get_race_results()
pgl = pgl[(pgl['year'] == year) & (pgl['round'] == rnd)][['driver', 'grid', 'position']]
pgl['change'] = pgl['grid'] - pgl['position']

def _color_red_or_green(val):
    color = 'orangered' if val < 0 else 'springgreen'
    return 'background-color: %s' % color
#pgl.style.applymap(_color_red_or_green)
#pgl.style.applymap(lambda x: "background-color: red" if x>0 else "background-color: white")
pgl.style.applymap(_color_red_or_green, subset=['change'])

# Average age of drivers per year

In [ ]:
raceresults = \
    pd.merge(
             pd.merge(pd.merge(stats.dfs['results'][['resultId', 'raceId', 'driverId', 'constructorId', 
                                                    'grid','position']], 
                               stats.dfs['races'][['raceId', 'year', 'round', 'name', 'date']]. \
                                       rename(columns={'name': 'race'})),
                      stats.dfs['drivers'][['driverId', 'number', 'name', 'dob']]. \
                              rename(columns={'name': 'driver'})),
             stats.dfs['constructors'][['constructorId', 'name']].rename(columns={'name': 'constructor'})
            )
raceresults = raceresults.sort_values(['year', 'round', 'position'])
raceresults['age'] = (pd.to_datetime(raceresults['date']).dt.date - raceresults['dob']) / np.timedelta64(1, 'Y')
avg_age = raceresults[['year', 'age']].groupby('year').mean()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

avg_age.plot(ax=ax)
ax.set_ylim(25,40)
ax.set_title('Average age per year')
ax.set_ylabel('age')
ax.set_xlabel('year')

# World champion with most DNF's

In [ ]:
dnfs = stats.get_dnfs()
dnfs = dnfs[['year', 'driverId', 'position']].groupby(['year', 'driverId']).count(). \
                                              reset_index().rename(columns={'position':'dnfs'})

champs = stats.get_wdc_champions()
champs = champs.merge(dnfs, on=['year', 'driverId'], how='left').fillna(0)
champs['dnf_perc'] = (champs['dnfs'] / champs['round'] * 1000).astype(int) / 10

champs[['year', 'name', 'wins', 'dnfs', 'dnf_perc']].sort_values('dnfs', ascending = False).head(10)

# Head to Head

In [ ]:
year = 2022
driver1 = 'VER'
driver2 = 'PER'

In [ ]:
race_results = stats.get_race_results()
race_results = race_results[race_results['year'] == year]

dr = [ race_results[race_results.code == driver1]['driverId'].values[0],
       race_results[race_results.code == driver2]['driverId'].values[0] ]

In [ ]:
race_results['position'] = race_results['position'].replace(0, 30)
race=[0, 0]
for rid in race_results.raceId.unique():
    pos1 = race_results.loc[(race_results.raceId == rid) & (race_results.driverId == dr[0]), 'position'].values[0]
    pos2 = race_results.loc[(race_results.raceId == rid) & (race_results.driverId == dr[1]), 'position'].values[0]
    if pos1 < pos2:
        race[0] += 1
    elif pos2 < pos1:
        race[1] += 1

In [ ]:
quali_results = stats.get_quali_results()
quali_results = quali_results[quali_results['year'] == year]
quali=[0, 0]
for rid in quali_results.raceId.unique():
    pos1 = quali_results.loc[(quali_results.raceId == rid) & (quali_results.driverId == dr[0]), 'position'].values[0]
    pos2 = quali_results.loc[(quali_results.raceId == rid) & (quali_results.driverId == dr[1]), 'position'].values[0]
    if pos1 < pos2:
        quali[0] += 1
    elif pos2 < pos1:
        quali[1] += 1

In [ ]:
standings = stats.get_wdc_standing()
standings = standings[(standings['year'] == year)]
standings = standings[(standings['round'] == standings['round'].max())]
pnts = [ standings.loc[standings.driverId == dr[0], 'points'].values[0],
         standings.loc[standings.driverId == dr[1], 'points'].values[0]  ]

In [ ]:
bpos = [ race_results.loc[race_results['driverId'] == dr[0], 'position'].min(),
         race_results.loc[race_results['driverId'] == dr[1], 'position'].min() ]

In [ ]:
bgr  = [ race_results.loc[race_results['driverId'] == dr[0], 'grid'].min(),
         race_results.loc[race_results['driverId'] == dr[1], 'grid'].min() ]

In [ ]:
dnfs = stats.get_dnfs()
dnfs = dnfs[dnfs['year'] == year]
dnf = [ len(dnfs[dnfs.driverId == dr[0]]), 
        len(dnfs[dnfs.driverId == dr[1]]) ]

In [ ]:
wins = [ len(race_results.loc[(race_results['driverId'] == dr[0]) & (race_results['position'] == 1), 'position']),
         len(race_results.loc[(race_results['driverId'] == dr[1]) & (race_results['position'] == 1), 'position']) ]

podia = [ len(race_results.loc[(race_results['driverId'] == dr[0]) & (race_results['position'] <= 3), 'position']),
          len(race_results.loc[(race_results['driverId'] == dr[1]) & (race_results['position'] <= 3), 'position']) ]

In [ ]:
print('Driver         : {}   {}'.format(driver1, driver2))
print('Driver IDs     : {:3d}   {:3d}'.format(dr[0], dr[1]))
print('Race           : {:3d}   {:3d}'.format(race[0], race[1]))
print('Qualifying     : {:3d}   {:3d}'.format(quali[0], quali[1]))
print('Points         : {:3.0f}   {:3.0f}'.format(pnts[0], pnts[1]))
print('Best finsish   : {:3d}   {:3d}'.format(bpos[0], bpos[1]))
print('Best grid      : {:3d}   {:3d}'.format(bgr[0], bgr[1]))
print('DNFs           : {:3d}   {:3d}'.format(dnf[0], dnf[1]))
print('Wins           : {:3d}   {:3d}'.format(wins[0], wins[1]))
print('Podiums        : {:3d}   {:3d}'.format(podia[0], podia[1]))

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

driver_clrs = [stats.get_driver_color(dr[0]), stats.get_driver_color(dr[1]) ]

ys    = ['Race', 'Race',
         'Quali', 'Quali',
         'Points', 'Points', 
         'Best finish', 'Best finish', 
         'Best grid', 'Best grid', 
         'DNF', 'DNF']
xmaxs = [-race[0]/max(race), race[1]/max(race), \
         -quali[0]/max(quali), quali[1]/max(quali), \
         -pnts[0]/max(pnts), pnts[1]/max(pnts), \
         -bpos[0]/max(bpos), bpos[1]/max(bpos), \
         -bgr[0]/max(bgr), bgr[1]/max(bgr), \
         -dnf[0]/max(dnf), dnf[1]/max(dnf)]
labels = [race[0], race[1], \
          quali[0], quali[1], \
          pnts[0], pnts[1], \
          bpos[0], bpos[1], \
          bgr[0], bgr[1], \
          dnf[0], dnf[1]]
clrs  = [driver_clrs[0], driver_clrs[1], 
         driver_clrs[0], driver_clrs[1],  
         driver_clrs[0], driver_clrs[1],  
         driver_clrs[0], driver_clrs[1],  
         driver_clrs[0], driver_clrs[1],  
         driver_clrs[0], driver_clrs[1], ]

rects = ax.hlines(y=ys, xmin=0, xmax=xmaxs, color=clrs, alpha=0.4, linewidth=35)
for nm, lbl, vl in zip(ys, labels, xmaxs):
    label = ax.annotate(str(int(lbl)), xy=(vl, nm), xytext=(5 if vl > 0 else -20, 0),
                        textcoords="offset points",
                        ha="left", va='center')

ax.set_xlim(-1.25, 1.25)
ax.invert_yaxis()
ax.axvline(0, color='black')
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
ax.set_title('Comparison {} to {}'.format(driver1, driver2))

In [ ]:
fig, ax = plt.subplots(figsize=(8,3.5))

driver_clrs = [stats.get_driver_color(dr[0]), stats.get_driver_color(dr[1]) ]

ys    = ['Points', 'Points', 
         'Wins', 'Wins', 
         'Podiums', 'Podiums', 
         'Quali', 'Quali', 
         'Race', 'Race']
xmaxs = [-pnts[0]/max(pnts), pnts[1]/max(pnts), \
         -wins[0]/max(wins), wins[1]/max(wins), \
         -podia[0]/max(podia), podia[1]/max(podia), \
         -quali[0]/max(quali), quali[1]/max(quali), \
         -race[0]/max(race), race[1]/max(race)]
labels = [pnts[0], pnts[1], \
          wins[0], wins[1], \
          podia[0], podia[1], \
          quali[0], quali[1], \
          race[0], race[1]]
clrs  = [driver_clrs[0], driver_clrs[1], 
         driver_clrs[0], driver_clrs[1],  
         driver_clrs[0], driver_clrs[1],  
         driver_clrs[0], driver_clrs[1],  
         driver_clrs[0], driver_clrs[1], ]

rects = ax.hlines(y=ys, xmin=0, xmax=xmaxs, color=clrs, alpha=0.4, linewidth=35)
for nm, lbl, vl in zip(ys, labels, xmaxs):
    label = ax.annotate(str(int(lbl)), xy=(vl, nm), xytext=(5 if vl > 0 else -20, 0),
                        textcoords="offset points",
                        ha="left", va='center')

ax.set_xlim(-1.25, 1.25)
ax.axvline(0, color='black')
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
ax.set_title('Comparison {} to {}'.format(driver1, driver2))

# Compare lap times

In [ ]:
season = 2022
rnd = 13
driver1 = 'VER'
driver2 = 'LEC'
laps = stats.get_laps_for_race(season, rnd)
laps = laps[laps['code'].isin([driver1, driver2])]
laps['milliseconds'] = laps['milliseconds'].div(1000)

In [ ]:
horizontal_driver_lines_plot(df=laps, xcolumn='lap', ycolumn='milliseconds', invert=False, ymin=80, ymax=120, 
                             title='Comparing lap times', xlabel='lap', ylabel='lap time')

# Most dominated seasons

In [ ]:
rnd = 13

results = stats.get_race_results()
results

points = [0, 25, 18, 15, 12, 10, 8, 6, 4, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
results['points'] = results['position'].apply(lambda x: points[x])
results = results[results['round'] <= rnd]
results = results[['year', 'driverId', 'driver', 'constructor', 'points']]. \
                 groupby(['year', 'driverId', 'driver', 'constructor']).sum().reset_index()
results['points'] = results['points'].div(rnd)
results = results.set_index('year').sort_values('points', ascending=False).groupby('year').nth(0). \
                  sort_values('points', ascending=False).reset_index()
results['desc'] = results['year'].astype(str) + ' ' + results['driver']
results

In [ ]:
horizontal_barplot(df=results, rowcount=10, sort_value='points', sort_ascending=False, 
                   ycolumn='points', xcolumn='desc', 
                   title='Most dominated sesaons (points according current system)', 
                   xlabel='points', ylabel='')

# Most laps led since 2020

In [ ]:
year = 2020

races = stats.dfs['races']
races = races[races['year'] >= year]
raceids = races['raceId'].values

laps_led = stats.dfs['lap_times']
laps_led = laps_led[laps_led['raceId'].isin(raceids)]
laps_led = laps_led[['driverId', 'lap']].groupby('driverId').count().reset_index()
laps_led = laps_led.merge(stats.dfs['drivers'][['driverId', 'code', 'name']])
laps_led

In [ ]:
horizontal_barplot(df=laps_led, rowcount=10, sort_value='lap', sort_ascending=False, 
                   xcolumn='name', ycolumn='lap', title='Most laps led since {}'.format(year), 
                   xlabel = 'laps', ylabel='', invert_yaxis=True)

# Points scored in last 6 rounds

In [ ]:
results = stats.get_race_results()
results = results[results['year'] == results['year'].max()]
results = results[(results['round'] <= results['round'].max()) & (results['round'] > results['round'].max()-6)]
results = results[['driver', 'code', 'points']].groupby(['driver', 'code']).sum().reset_index(). \
              rename(columns={'driver' : 'name'}).sort_values('points', ascending=False)

horizontal_barplot(df=results, rowcount=10, sort_value='points', sort_ascending=False, 
                   xcolumn='name', ycolumn='points', title='Points scored in last 6 rounds', 
                   xlabel = 'points', ylabel='', invert_yaxis=True)

In [ ]:


horizontal_driver_lines_plot(df=standings, xcolumn='round', ycolumn='position', invert_yaxis=True,
                             title='Championships standings ({})'.format(year),
                             xlabel='Round', ylabel='Position')